In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta
import time
from itertools import chain, product

In [4]:
# Default settings
pd.options.display.max_columns = None
%matplotlib inline
sns.set(style="darkgrid")
random_seed = 42

In [5]:
df = pd.read_csv('data/Export_Verbruik_2010-2019_anon.csv', index_col=0)
df.head()

,VERBRUIK_ID,VERBRUIK_OBJECT_ID,VERBRUIK_STARTDAT,VERBRUIK_EINDDATUM,VERBRUIK_GESCHAT_JN,VERBRUIK,POSTCODE,STAD,OBJECT_TYPE_NAME
0,0x7E405BC6FDD0BB06E1F7711B39D7C3CB684FAA20,0xB3CC642C47DA2964C780FC5590DEAB3FAA791F9D,2013-03-08,2013-10-07,N,28.0,1013,AMSTERDAM,HHB
1,0xD763B3D7286E78B4102C06DF1FE478C899B38743,0xB3CC642C47DA2964C780FC5590DEAB3FAA791F9D,2013-10-07,2014-10-06,J,48.0,1013,AMSTERDAM,HHB
2,0xD72C616CCDAAA45E049AF4AB3B5F92DD853B394A,0xB3CC642C47DA2964C780FC5590DEAB3FAA791F9D,2014-10-06,2014-12-05,J,8.0,1013,AMSTERDAM,HHB
3,0x9B04DD93592300582B286C693DE79E64AA474DF5,0xB3CC642C47DA2964C780FC5590DEAB3FAA791F9D,2014-12-05,2015-11-06,N,37.0,1013,AMSTERDAM,HHB
4,0x04F71377870507FA1CFC31EC9D372D241AF1CF80,0xB3CC642C47DA2964C780FC5590DEAB3FAA791F9D,2015-11-06,2016-10-13,N,52.0,1013,AMSTERDAM,HHB


In [6]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4203393 entries, 0 to 4203392
Data columns (total 9 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   VERBRUIK_ID          4203393 non-null  object 
 1   VERBRUIK_OBJECT_ID   4203393 non-null  object 
 2   VERBRUIK_STARTDAT    4203358 non-null  object 
 3   VERBRUIK_EINDDATUM   4203393 non-null  object 
 4   VERBRUIK_GESCHAT_JN  4203393 non-null  object 
 5   VERBRUIK             4203393 non-null  float64
 6   POSTCODE             4203393 non-null  int64  
 7   STAD                 4192751 non-null  object 
 8   OBJECT_TYPE_NAME     4201632 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 320.7+ MB


In [7]:
df.describe()

,VERBRUIK,POSTCODE
count,4.203393e+06,4.203393e+06
mean,1.698459e+02,1.111630e+03
std,6.532013e+03,2.057869e+02
min,-4.729757e+06,0.000000e+00
25%,3.000000e+01,1.051000e+03
50%,6.500000e+01,1.071000e+03
75%,1.180000e+02,1.103000e+03
max,1.497390e+06,2.106000e+03


In [8]:
df['POSTCODE'].value_counts(dropna=False)

1181    118523
1102    112007
1013    111993
1019    107198
1069    107114
         ...  
1027      1987
1041      1774
1114      1720
1391       828
1037       664
Name: POSTCODE, Length: 104, dtype: int64

In [9]:
# checking anonimized postal codes
df.loc[df['POSTCODE'] == 0]['POSTCODE'].value_counts()

0    14513
Name: POSTCODE, dtype: int64

In [10]:
# we find some missing values, it seem to be only 35 rows so lets remove there
df = df.loc[df['VERBRUIK_STARTDAT'].notna()]
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4203358 entries, 0 to 4203392
Data columns (total 9 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   VERBRUIK_ID          4203358 non-null  object 
 1   VERBRUIK_OBJECT_ID   4203358 non-null  object 
 2   VERBRUIK_STARTDAT    4203358 non-null  object 
 3   VERBRUIK_EINDDATUM   4203358 non-null  object 
 4   VERBRUIK_GESCHAT_JN  4203358 non-null  object 
 5   VERBRUIK             4203358 non-null  float64
 6   POSTCODE             4203358 non-null  int64  
 7   STAD                 4192718 non-null  object 
 8   OBJECT_TYPE_NAME     4201597 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 320.7+ MB


In [11]:
# checking object types
df['OBJECT_TYPE_NAME'].value_counts()

HHB                         3890819
KZB                          279202
GZB                           28818
IND                            2155
ENG                             396
OWT                             101
SIER                             77
Geen (nvt voor Waternet)         15
Overig                           12
BRA                               2
Name: OBJECT_TYPE_NAME, dtype: int64

In [12]:
# removing object types that are not in scope
scope = ['HHO', 'HHB', 'GZB', 'KZB', 'KZO']
df = df.loc[df['OBJECT_TYPE_NAME'].isin(scope)]
df['OBJECT_TYPE_NAME'].value_counts()

HHB    3890819
KZB     279202
GZB      28818
Name: OBJECT_TYPE_NAME, dtype: int64

In [13]:
# we also find some rows with no water usage, so removing those too
print(df.loc[df['VERBRUIK'] == 0].shape)
df = df.loc[df['VERBRUIK'] != 0]

(191357, 9)


In [14]:
# coverting date-like columns to datetime
df[['VERBRUIK_STARTDAT', 'VERBRUIK_EINDDATUM']] = df[['VERBRUIK_STARTDAT', 'VERBRUIK_EINDDATUM']].apply(pd.to_datetime)
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4007482 entries, 0 to 4203392
Data columns (total 9 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   VERBRUIK_ID          4007482 non-null  object        
 1   VERBRUIK_OBJECT_ID   4007482 non-null  object        
 2   VERBRUIK_STARTDAT    4007482 non-null  datetime64[ns]
 3   VERBRUIK_EINDDATUM   4007482 non-null  datetime64[ns]
 4   VERBRUIK_GESCHAT_JN  4007482 non-null  object        
 5   VERBRUIK             4007482 non-null  float64       
 6   POSTCODE             4007482 non-null  int64         
 7   STAD                 3998834 non-null  object        
 8   OBJECT_TYPE_NAME     4007482 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(5)
memory usage: 305.7+ MB


In [15]:
df['VERBRUIK_STARTDAT'].dt.year.value_counts()

2018    440062
2016    422932
2010    419402
2015    414178
2017    402392
2014    397101
2011    380609
2013    380423
2012    377917
2009    303991
2019     61080
2008      5230
2007      1265
2006       418
2005       267
2004       114
2003        73
2002        25
2000         2
2001         1
Name: VERBRUIK_STARTDAT, dtype: int64

In [16]:
# calculate period
df['DIFF'] = (df['VERBRUIK_EINDDATUM'] - df['VERBRUIK_STARTDAT']).dt.days
df['DIFF']

0          213
1          364
2           60
3          336
4          342
          ... 
4203385    339
4203386    365
4203387    365
4203390    239
4203392    273
Name: DIFF, Length: 4007482, dtype: int64

In [17]:
df['AVG_DAY'] = df['VERBRUIK'] / df['DIFF']
df.head()

,VERBRUIK_ID,VERBRUIK_OBJECT_ID,VERBRUIK_STARTDAT,VERBRUIK_EINDDATUM,VERBRUIK_GESCHAT_JN,VERBRUIK,POSTCODE,STAD,OBJECT_TYPE_NAME,DIFF,AVG_DAY
0,0x7E405BC6FDD0BB06E1F7711B39D7C3CB684FAA20,0xB3CC642C47DA2964C780FC5590DEAB3FAA791F9D,2013-03-08,2013-10-07,N,28.0,1013,AMSTERDAM,HHB,213,0.131455
1,0xD763B3D7286E78B4102C06DF1FE478C899B38743,0xB3CC642C47DA2964C780FC5590DEAB3FAA791F9D,2013-10-07,2014-10-06,J,48.0,1013,AMSTERDAM,HHB,364,0.131868
2,0xD72C616CCDAAA45E049AF4AB3B5F92DD853B394A,0xB3CC642C47DA2964C780FC5590DEAB3FAA791F9D,2014-10-06,2014-12-05,J,8.0,1013,AMSTERDAM,HHB,60,0.133333
3,0x9B04DD93592300582B286C693DE79E64AA474DF5,0xB3CC642C47DA2964C780FC5590DEAB3FAA791F9D,2014-12-05,2015-11-06,N,37.0,1013,AMSTERDAM,HHB,336,0.110119
4,0x04F71377870507FA1CFC31EC9D372D241AF1CF80,0xB3CC642C47DA2964C780FC5590DEAB3FAA791F9D,2015-11-06,2016-10-13,N,52.0,1013,AMSTERDAM,HHB,342,0.152047


In [18]:
grouped = df[['OBJECT_TYPE_NAME', 'POSTCODE', 'VERBRUIK_STARTDAT', 'VERBRUIK_EINDDATUM', 'AVG_DAY']].groupby(['OBJECT_TYPE_NAME', 'POSTCODE', 'VERBRUIK_STARTDAT', 'VERBRUIK_EINDDATUM']).sum().reset_index()
grouped

,OBJECT_TYPE_NAME,POSTCODE,VERBRUIK_STARTDAT,VERBRUIK_EINDDATUM,AVG_DAY
0,GZB,0,2009-12-22,2010-01-27,171.916667
1,GZB,0,2009-12-23,2010-01-26,51.882353
2,GZB,0,2009-12-28,2010-01-26,3182.448276
3,GZB,0,2010-01-26,2010-02-22,38.185185
4,GZB,0,2010-01-26,2010-02-23,3135.214286
...,...,...,...,...,...
2425104,KZB,2106,2019-11-19,2019-12-01,0.083333
2425105,KZB,2106,2019-11-22,2019-11-30,1.000000
2425106,KZB,2106,2019-11-22,2019-12-01,0.111111
2425107,KZB,2106,2019-11-30,2019-12-01,-4.000000


In [30]:
# unpivot grouped dataframe, resulting into two measurements for each row (one for start and one for end date)
melt = grouped.reset_index().melt(id_vars=['index', 'OBJECT_TYPE_NAME', 'POSTCODE', 'AVG_DAY'], value_name='DATE').drop('variable', axis=1)
melt['DATE'] = pd.to_datetime(melt['DATE'])
melt

,index,OBJECT_TYPE_NAME,POSTCODE,AVG_DAY,DATE
0,0,GZB,0,171.916667,2009-12-22
1,1,GZB,0,51.882353,2009-12-23
2,2,GZB,0,3182.448276,2009-12-28
3,3,GZB,0,38.185185,2010-01-26
4,4,GZB,0,3135.214286,2010-01-26
...,...,...,...,...,...
4850213,2425104,KZB,2106,0.083333,2019-12-01
4850214,2425105,KZB,2106,1.000000,2019-11-30
4850215,2425106,KZB,2106,0.111111,2019-12-01
4850216,2425107,KZB,2106,-4.000000,2019-12-01


In [31]:
# subset for faster testing purposes
melt = melt.loc[melt['index'] <= 1000]
melt

,index,OBJECT_TYPE_NAME,POSTCODE,AVG_DAY,DATE
0,0,GZB,0,171.916667,2009-12-22
1,1,GZB,0,51.882353,2009-12-23
2,2,GZB,0,3182.448276,2009-12-28
3,3,GZB,0,38.185185,2010-01-26
4,4,GZB,0,3135.214286,2010-01-26
...,...,...,...,...,...
2426105,996,GZB,1016,79.833333,2016-10-26
2426106,997,GZB,1016,79.076923,2016-11-21
2426107,998,GZB,1016,65.571429,2016-12-19
2426108,999,GZB,1016,87.990868,2017-07-26


In [32]:
start = time.time()
# grouping by index and filling dates between start and end dates
melt = melt.groupby('index').apply(lambda x: x.set_index('DATE').resample('D').first())\
           .ffill()\
           .reset_index(level=1)\
           .reset_index(drop=True)

print(time.time() - start)
melt

5.238324880599976


,DATE,index,OBJECT_TYPE_NAME,POSTCODE,AVG_DAY
0,2009-12-22,0.0,GZB,0.0,171.916667
1,2009-12-23,0.0,GZB,0.0,171.916667
2,2009-12-24,0.0,GZB,0.0,171.916667
3,2009-12-25,0.0,GZB,0.0,171.916667
4,2009-12-26,0.0,GZB,0.0,171.916667
...,...,...,...,...,...
33104,2017-08-18,1000.0,GZB,1016.0,107.111111
33105,2017-08-19,1000.0,GZB,1016.0,107.111111
33106,2017-08-20,1000.0,GZB,1016.0,107.111111
33107,2017-08-21,1000.0,GZB,1016.0,107.111111
